<a href="https://colab.research.google.com/github/BayaderJ/Data-Exploration-Interactive-Dashboard-Project/blob/main/tomtomtry.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
"""
MAX Riyadh Metro Data TODAY (2,500 calls) - FULL DEBUG MODE
3 routes × 24h × ~29 days = 2,088 calls
Shows EVERY API response + live progress
"""

import requests
import pandas as pd
import datetime as dt
from time import sleep
import json

# CONFIG
TOMTOM_API_KEY = "tKz67coKH9RenXelL3V3nHPLWLEkszPP"
ROUTES = [
    {"route_id": "R1_blue_olaya", "start_lat": 24.7700, "start_lon": 46.6700, "end_lat": 24.6350, "end_lon": 46.7150},
    {"route_id": "R2_red_king_abdullah", "start_lat": 24.7200, "start_lon": 46.6000, "end_lat": 24.7300, "end_lon": 46.8200},
    {"route_id": "R3_green_king_abdulaziz", "start_lat": 24.7950, "start_lon": 46.7100, "end_lat": 24.6500, "end_lon": 46.7300},
]
START_DATE = dt.date(2024, 1, 1)
END_DATE = dt.date(2025, 10, 31)
HOURS = list(range(24))
SLEEP_SECONDS = 0.5
OUTPUT_CSV = "riyadh_metro_max_today.csv"
MAX_CALLS = 2400  # Safety limit

print("🚀 STARTING Riyadh Metro Traffic Collection")
print(f"📊 Plan: 3 routes × 24h × ~29 days = {3*24*29} rows")
print(f"⏱️  ETA: ~2-3 hours | Safety limit: {MAX_CALLS} calls")
print("-" * 60)

def build_departure_iso(date_obj, hour):
    dt_obj = dt.datetime(date_obj.year, date_obj.month, date_obj.day, hour, 0, 0)
    return dt_obj.strftime("%Y-%m-%dT%H:%M:%S+03:00")

def use_this_day(d):
    if d.year == 2025 and d.month == 10:  # Full Oct 2025
        return True
    return d.day % 3 == 1

def get_route_summary(start_lat, start_lon, end_lat, end_lon, depart_iso, route_id, total_calls):
    """FULL DEBUG: Shows every API response"""
    url = f"https://api.tomtom.com/routing/1/calculateRoute/{start_lat},{start_lon}:{end_lat},{end_lon}/json"
    params = {"key": TOMTOM_API_KEY, "traffic": "true", "departAt": depart_iso}

    print(f"🔄 [{total_calls:04d}/{MAX_CALLS}] {route_id} | {depart_iso[:16]}", end=" → ")

    try:
        resp = requests.get(url, params=params, timeout=20)
        print(f"HTTP {resp.status_code}", end=" → ")

        if resp.status_code != 200:
            print("❌ ERROR")
            return None

        data = resp.json()
        if "routes" not in data or not data["routes"]:
            print("❌ No routes")
            return None

        summary = data["routes"][0]["summary"]
        travel_time = summary.get("travelTimeInSeconds", 0)
        length_m = summary.get("lengthInMeters", 0)
        speed_kmh = round((length_m / 1000) / (travel_time / 3600), 1) if travel_time > 0 else 0

        print(f"✅ {travel_time/60:.0f}min | {speed_kmh}km/h")
        return {
            "travel_time_s": travel_time,
            "length_m": length_m,
            "speed_kmh": speed_kmh
        }

    except Exception as e:
        print(f"💥 {str(e)[:30]}...")
        return None

# MAIN LOOP
all_rows = []
current_date = START_DATE
total_calls = 0
processed_days = 0

print(f"📅 First date to process: {current_date}")

while current_date <= END_DATE and total_calls < MAX_CALLS:
    if not use_this_day(current_date):
        current_date += dt.timedelta(days=1)
        continue

    print(f"\n📅 Processing {current_date} (Day {processed_days+1})")

    day_rows = 0
    for hour in HOURS:
        depart_iso = build_departure_iso(current_date, hour)
        for route in ROUTES:
            if total_calls >= MAX_CALLS:
                print(f"🛑 DAILY LIMIT REACHED: {total_calls}/{MAX_CALLS}")
                break

            summary = get_route_summary(
                route["start_lat"], route["start_lon"],
                route["end_lat"], route["end_lon"],
                depart_iso, route["route_id"], total_calls + 1
            )

            total_calls += 1
            sleep(SLEEP_SECONDS)

            if summary:
                all_rows.append({
                    "route_id": route["route_id"],
                    "date": current_date.isoformat(),
                    "hour": hour,
                    "travel_time_s": summary["travel_time_s"],
                    "length_m": summary["length_m"],
                    "speed_kmh": summary["speed_kmh"]
                })
                day_rows += 1

        if total_calls >= MAX_CALLS:
            break

    processed_days += 1
    print(f"✅ Day complete: {day_rows} rows | Total: {len(all_rows)} rows | {total_calls} calls")

    # Progress summary every 5 days
    if processed_days % 5 == 0:
        print(f"📈 PROGRESS: {processed_days} days | {len(all_rows)} rows | {total_calls}/{MAX_CALLS} calls")

# FINAL SAVE
print("\n" + "="*60)
print("💾 SAVING DATA...")
df = pd.DataFrame(all_rows)
df.to_csv(OUTPUT_CSV, index=False)
size_mb = len(df) * 120 / (1024*1024)

print(f"🎉 COMPLETE!")
print(f"📊 {len(df):,} rows saved to {OUTPUT_CSV}")
print(f"💾 Size: ~{size_mb:.1f} MB")
print(f"📞 Total API calls: {total_calls}/{MAX_CALLS}")
print(f"📅 Days processed: {processed_days}")
print(f"🔬 Perfect for ML prediction + metro analysis!")
print("\nNext: df = pd.read_csv('riyadh_metro_max_today.csv')")


🚀 STARTING Riyadh Metro Traffic Collection
📊 Plan: 3 routes × 24h × ~29 days = 2088 rows
⏱️  ETA: ~2-3 hours | Safety limit: 2400 calls
------------------------------------------------------------
📅 First date to process: 2024-01-01

📅 Processing 2024-01-01 (Day 1)
🔄 [0001/2400] R1_blue_olaya | 2024-01-01T00:00 → HTTP 200 → ✅ 25min | 49.4km/h
🔄 [0002/2400] R2_red_king_abdullah | 2024-01-01T00:00 → HTTP 200 → ✅ 28min | 63.6km/h
🔄 [0003/2400] R3_green_king_abdulaziz | 2024-01-01T00:00 → HTTP 200 → ✅ 24min | 63.0km/h
🔄 [0004/2400] R1_blue_olaya | 2024-01-01T01:00 → HTTP 200 → ✅ 25min | 49.4km/h
🔄 [0005/2400] R2_red_king_abdullah | 2024-01-01T01:00 → HTTP 200 → ✅ 28min | 63.6km/h
🔄 [0006/2400] R3_green_king_abdulaziz | 2024-01-01T01:00 → HTTP 200 → ✅ 24min | 63.0km/h
🔄 [0007/2400] R1_blue_olaya | 2024-01-01T02:00 → HTTP 200 → ✅ 25min | 49.4km/h
🔄 [0008/2400] R2_red_king_abdullah | 2024-01-01T02:00 → HTTP 200 → ✅ 28min | 63.6km/h
🔄 [0009/2400] R3_green_king_abdulaziz | 2024-01-01T02:00 → HT